# This notebook is written by Nilotpal Mrinal and is influenced by the paper by Geoffrey Hinton. This project in tensorflow is Also known as the hello world of tensorflow.

All the basic implementation and prediction have been done in this notebook. 

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Build a neural network that classifies images.
2. Train this neural network.
3. And, finally, evaluate the accuracy of the model.

This is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook file. 

In [17]:
import tensorflow as tf

In [6]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

Build the `tf.keras.Sequential` model by stacking layers. Choose an optimizer and loss function for training:

In [7]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [8]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.21239349, -0.02984576,  1.5874751 , -0.17799126, -0.32656175,
         0.04270613,  0.04641357,  0.30226347, -0.07777032, -0.04291008]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [9]:
tf.nn.softmax(predictions).numpy()

array([[0.08843087, 0.0694066 , 0.34977943, 0.0598497 , 0.05158681,
        0.07462935, 0.07490654, 0.09674621, 0.06615876, 0.06850574]],
      dtype=float32)

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [10]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.log(1/10) ~= 2.3`.

In [11]:
loss_fn(y_train[:1], predictions).numpy()

2.5952215

In [12]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [13]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
60000/60000 [==============================] - 5s 81us/sample - loss: 0.2886 - accuracy: 0.9166
Epoch 2/5
60000/60000 [==============================] - 4s 60us/sample - loss: 0.1377 - accuracy: 0.9596
Epoch 3/5
60000/60000 [==============================] - 3s 53us/sample - loss: 0.1042 - accuracy: 0.9682
Epoch 4/5
60000/60000 [==============================] - 3s 58us/sample - loss: 0.0836 - accuracy: 0.9740
Epoch 5/5
60000/60000 [==============================] - 4s 72us/sample - loss: 0.0723 - accuracy: 0.9765


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [14]:
model.evaluate(x_test,  y_test, verbose=2)

10000/10000 - 0s - loss: 0.0754 - accuracy: 0.9781


[0.07543381911152974, 0.9781]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [15]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [16]:
probability_model(x_test[:5])

<tf.Tensor: id=78130, shape=(5, 10), dtype=float32, numpy=
array([[5.8589715e-09, 2.8470093e-10, 2.3331243e-07, 5.7236208e-05,
        3.8156155e-11, 4.4989886e-08, 4.1336737e-15, 9.9993896e-01,
        9.9744915e-09, 3.4508171e-06],
       [1.4925028e-08, 2.1456071e-05, 9.9996114e-01, 4.5389788e-06,
        1.6118735e-15, 1.2686263e-05, 2.4410096e-07, 2.1790333e-13,
        9.3888728e-09, 1.1670646e-13],
       [4.9667970e-06, 9.9910283e-01, 2.5371657e-05, 5.1972300e-05,
        1.8704175e-05, 3.2348598e-05, 8.5091655e-05, 6.0124282e-04,
        7.6141216e-05, 1.3788314e-06],
       [9.9962592e-01, 3.7785824e-08, 8.3678126e-05, 1.8099753e-07,
        9.7243210e-06, 1.5867261e-05, 6.4769141e-05, 4.0766354e-06,
        9.3906721e-08, 1.9552004e-04],
       [1.3126217e-05, 4.8859352e-08, 5.0719277e-06, 6.0188086e-08,
        9.9786609e-01, 1.6411292e-07, 1.3823285e-06, 8.8113869e-05,
        5.1340368e-07, 2.0254229e-03]], dtype=float32)>